In [1]:
# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
tf.__version__

'2.17.0'

In [3]:
#preprocessing the training set
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,
                                   zoom_range = 0.2,horizontal_flip = True)
training_set = train_datagen.flow_from_directory('datasets/training_set',target_size = (64, 64),
                                                 batch_size = 32,class_mode = 'binary')

Found 8000 images belonging to 2 classes.


In [4]:
#preprocessing test dataset
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('datasets/test_set',target_size = (64, 64),
                                            batch_size = 32,class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [5]:
cnn = tf.keras.models.Sequential()

# Convolution Layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

#Pooling Layer
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#Flatenning
cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))

#Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

d:\ML_ENV\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

import datetime
early_stopping = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

cnn.fit(x = training_set, validation_data = test_set, epochs = 25, callbacks=[early_stopping, tensorboard_callback])

Epoch 1/25


d:\ML_ENV\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 242s 891ms/step - accuracy: 0.5078 - loss: 0.7158 - val_accuracy: 0.5890 - val_loss: 0.6653
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 30s 117ms/step - accuracy: 0.5998 - loss: 0.6651 - val_accuracy: 0.6505 - val_loss: 0.6516
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 30s 116ms/step - accuracy: 0.6367 - loss: 0.6424 - val_accuracy: 0.6860 - val_loss: 0.5912
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 115ms/step - accuracy: 0.6755 - loss: 0.6032 - val_accuracy: 0.7065 - val_loss: 0.5728
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 30s 116ms/step - accuracy: 0.7002 - loss: 0.5812 - val_accuracy: 0.7265 - val_loss: 0.5576
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 114ms/step - accuracy: 0.7244 - loss: 0.5515 - val_accuracy: 0.7185 - val_loss: 0.5558
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 115ms/step - accuracy: 0.7340 - loss: 0.5332 - val_accuracy: 0.7225 - val_loss: 0.5383
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 30s 116ms/step - accuracy: 0.7524 - loss: 0.5176 - va

In [7]:
'''
import numpy as np
from tensorflow.keras.preprocessing import image


test_image = image.load_img('PATH_TO_FILE', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

'''


"\nimport numpy as np\nfrom tensorflow.keras.preprocessing import image\n\n\ntest_image = image.load_img('PATH_TO_FILE', target_size = (64, 64))\ntest_image = image.img_to_array(test_image)\ntest_image = np.expand_dims(test_image, axis = 0)\nresult = cnn.predict(test_image)\ntraining_set.class_indices\nif result[0][0] == 1:\n  prediction = 'dog'\nelse:\n  prediction = 'cat'\n\n"

In [8]:
cnn.save("models/cnn.keras")